In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
# 라운지 가상 사용자 파일 가져오기 / 구성: 사용자	위치	시간	사용자 수	책상 높이	콘센트 필요도	컴퓨터 필요도	정수기/화장실 유무
raw_data_user = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지 사용자 5000 최종.csv',encoding='euc-kr')

#	가상 라운지 파일 가져오기 / 구성: 라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도  건물
raw_data_lounge = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/라운지 데이터 최종.csv',encoding='euc-kr')
raw_data_lounge['건물'] = raw_data_lounge['건물'].replace({' ':''}, regex=True)

# 랜드마크 간의 거리 가져오기
distance_buildings = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/건물별 이동시간 최종.csv',encoding='utf-8')

# 혼잡도 파일 가져오기
con_0 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_월.csv')
con_1 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_화.csv')
con_2 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_수.csv')
con_3 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_목.csv')
con_4 = pd.read_csv('/content/drive/My Drive/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/라운지+혼잡도/혼잡도_금.csv')

In [ ]:
# 모든 라운지에 대한 각 사용자 별 점수 계산
def lounge_score(user,theme):

    user_building = user['위치']
    distance = distance_buildings[['building', user_building]].copy()
    distance.rename(columns = {user_building: '거리'}, inplace = True)

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도	건물	***거리
    lounge_with_distance = pd.merge(raw_data_lounge, distance, left_on= '건물', right_on='building', how ='left' ).drop(columns=['building'])

    day = str(pd.to_datetime(user['시간'].split('T')[0]).dayofweek)
    user_c = con_0

    #find day
    if day==1:
        user_c = con_1
    elif day==2:
        user_c = con_2
    elif day==3:
        user_c = con_3
    elif day==4:
        user_c = con_4

    range = pd.date_range("09:00", "23:30", freq="30min").strftime('%H:%M:%S')
    dt = pd.to_datetime(user['시간']).round('30min')
    user_c_r = range.get_loc(str(dt).split(' ')[1].split('+')[0])

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	***혼잡도	건물    거리
    final_lounge = lounge_with_distance
    final_lounge['혼잡도']=final_lounge.apply(lambda row: user_c[row.건물][user_c_r], axis=1)


    # 정규화
    '''
    norm lists :-
        1) desk_norm : 책상높이가 사용자 선호와 같으면 1
        2) facilities_norm : 정수기/화장실이 사용자 선호와 가까울수록 1 (하나당 -0.5)
        3) chair_norm : 의자 수 >= 사용자 수 면 1
        4) person_per_table_norm : 사용자 수와 책상별 앉을 수 있는 사람의 수가 비슷할수록 1
        5) plugs_norm : 콘센트 수 ~ 사용자 필요도
        6) computers_norm : 컴퓨터 수 ~ 사용자 필요도
        7) distance_norm : 거리가 멀수록 0 가까울수록 1
        8) congestion_norm : 혼잡할수록 0 가까울수록 1
        9) ac_norm : 있으면 1 없으면 0
        10) window_norm : 있으면 1 없으면 0
    '''
    satisfy_user_idx = final_lounge['책상 높이'] == user['책상 높이']
    desk_norm = final_lounge['라운지'].copy()
    desk_norm[satisfy_user_idx] = 1
    desk_norm[~satisfy_user_idx] = 0

    satisfy_user_idx = final_lounge['정수기'] == user['정수기/화장실 유무']
    facilities_norm = final_lounge['라운지'].copy()
    facilities_norm[satisfy_user_idx] = 0.5
    facilities_norm[~satisfy_user_idx] = 0.0
    satisfy_user_idx = final_lounge['화장실'] == user['정수기/화장실 유무']
    facilities_norm[satisfy_user_idx] += 0.5
    facilities_norm[~satisfy_user_idx] += 0.0

    satisfy_user_idx = final_lounge['의자 수'] >= user['사용자 수']
    chair_norm = final_lounge['라운지'].copy()
    chair_norm[satisfy_user_idx] = 1
    chair_norm[~satisfy_user_idx] = 0

    person_per_table_diff = abs(user['사용자 수'] - (final_lounge['의자 수']/final_lounge['테이블 수']))
    person_per_table_norm = 1 - (person_per_table_diff/(person_per_table_diff.max()-person_per_table_diff.min()))

    plugs = (final_lounge['콘센트 수']/final_lounge['의자 수'])*user['콘센트 필요도']
    plugs_norm = (plugs-plugs.min())/(plugs.max()-plugs.min())

    #멀수록 0 가까울수록 1
    distance_norm = 1 - (final_lounge['거리']-final_lounge['거리'].min())/(final_lounge['거리'].max()-final_lounge['거리'].min())

    #혼잡할수록 0 한가할수록 1
    congestion_norm = 1 - (final_lounge['혼잡도']-final_lounge['혼잡도'].min())/(final_lounge['혼잡도'].max()-final_lounge['혼잡도'].min())

    computers = (final_lounge['컴퓨터']/final_lounge['의자 수'])*user['컴퓨터 필요도']
    computers_norm = (computers-computers.min())/(computers.max()-computers.min())

    ac_norm = final_lounge['냉난방장치']

    window_norm = final_lounge['창문']

    # 테마 구분
    # 1. 휴식
    if theme == 1:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.68+computers_norm*0.115+distance_norm*4+congestion_norm*5+ac_norm*0.5+window_norm*0.1
    # 2. 대화
    elif theme == 2:
        scores = desk_norm*0.15+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.88+computers_norm*0.115+distance_norm*4+(1-congestion_norm)*3.5+ac_norm*0.5+window_norm*0.1
    # 3. 팀공부
    elif theme == 3:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.88+computers_norm*0.115+distance_norm*4+(1-congestion_norm)*3.5+ac_norm*0.5+window_norm*0.1
    # 0. 공부
    else:
        scores = desk_norm*0.71+facilities_norm*0.68+chair_norm+person_per_table_norm+plugs_norm*0.68+computers_norm*0.115+distance_norm*4+congestion_norm*3+ac_norm*0.5+window_norm*0.1

    return scores

In [ ]:
# 사용자의 라운지 점수(true label)에 대한 dataframe 생성
df_final = pd.DataFrame(columns=['user_index','score'])
for idx,row in raw_data_user.iterrows():
    print(row['사용자'])
    df_final = df_final.append({'user_index' : row['사용자'] , 'score' : lounge_score(row,3).tolist()} , ignore_index=True)

from google.colab import files

# 사용자의 라운지 점수(true label) 저장 
df_final.to_csv('라운지_트루라벨_3.csv', index = None,encoding='euc-kr') 
files.download('라운지_트루라벨_3.csv')